In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
model=ChatGoogleGenerativeAI(model='gemini-2.5-flash')
parser=StrOutputParser()

In [16]:
result=model.invoke('hy')

In [20]:
result

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--eac02db7-62b8-472f-8165-1326c3997e3e-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [21]:
result.content

'Hi there! How can I help you today?'

In [22]:
parser.invoke(result)

'Hi there! How can I help you today?'

In [ ]:
while True: 
    message=input("Write your query")
    if message=='end': 
        print('Good by have a great day')
        break
    else: 
        print(message, ":", parser.invoke(model.invoke([HumanMessage(content=message)])))


# here it is not keeping the track of memory so it is not a chat bot basicaly it is Q&a System without tracking the memory

hy  : Hey there! How can I help you today?
what is capital of india : The capital of India is **New Delhi**.
my name is pankaj , remember always : Okay, Pankaj. I will remember that. Welcome!
what will the output of the 2+2 : The output of 2+2 will be **4**.
what is my name : I don't know your name. As an AI, I don't have access to personal information about you or retain details from past conversations. My interactions are stateless, meaning each time you ask me something, it's like a fresh start.

If you'd like me to refer to you by a specific name during our chat, please feel free to tell me!
Good by have a great day


## 1 chat history manualy


In [ ]:

chat_history=[AIMessage(content="You are a helpful assistant")]

while True: 
    message=input('Write your query')
    chat_history.append(HumanMessage(content=message))
    if message=='end': 
        print('Good by have a great day')
        break
    else: 
        response=parser.invoke(model.invoke(chat_history))
        chat_history.append(AIMessage(content=response))
        print(f'HumanMessage: {message}\nAIMessage: {response}')



# this is the manual way to store chat history

HumanMessage: hy
AIMessage: Hello! How can I help you today?
HumanMessage: my name is pankaj kumawat
AIMessage: Nice to meet you, Pankaj! How can I help you today?
HumanMessage: what is the capital of india
AIMessage: The capital of India is **New Delhi**.
HumanMessage: what will the result of 2 + 2
AIMessage: The result of 2 + 2 is **4**.
HumanMessage: what is my name
AIMessage: Your name is **Pankaj Kumawat**.
HumanMessage: what was the last math operation
AIMessage: The last math operation we discussed was **addition** (specifically, 2 + 2).
HumanMessage: what is the population of the previouos city
AIMessage: The previous city we discussed was **New Delhi**.

The population of the **National Capital Territory (NCT) of Delhi**, which includes New Delhi, is estimated to be around **30 million** people (as of recent estimates for the broader metropolitan area).
Good by have a great day


## 2 Chat history using Predefined functions

In [37]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

| Component                        | Purpose                                                | Use                                                       |
| -------------------------------- | ------------------------------------------------------ | --------------------------------------------------------- |
| **`BaseChatMessageHistory`**     | Blueprint (interface) for storing chat history         | Used when creating a **custom** memory backend (like DB). |
| **`InMemoryChatMessageHistory`** | Stores chat history **temporarily in RAM**             | Used for simple or short-lived sessions.                  |
| **`RunnableWithMessageHistory`** | Wraps your chain/model to **auto-manage chat history** | Lets the model remember previous conversations.           |


👉 In short:<br>
BaseChatMessageHistory = rulebook<br>
InMemoryChatMessageHistory = ready-to-use memory<br>
RunnableWithMessageHistory = makes your chatbot remember chats <br>automatically ✅

In [38]:
store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory: 
    if session_id not in store: 
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

In [39]:
model_with_memory=RunnableWithMessageHistory(model, get_session_history)

In [40]:
config={'configurable': {'session_id': 'firstchat'}}
model_with_memory.invoke([HumanMessage(content='hy i am pankaj kumawat')], config=config).content

"Hi Pankaj, it's nice to meet you! How can I help you today?"

In [41]:
model_with_memory.invoke([HumanMessage(content='what is my name')], config=config).content

'Your name is **Pankaj Kumawat**. You told me that in your first message!'

In [42]:
config = {"configurable": {"session_id": "secondtchat"}}
model_with_memory.invoke([HumanMessage(content='what is my name')], config=config).content

"I don't know your name. As an AI, I don't have access to personal information about you unless you choose to tell me."

In [43]:
config={'configurable': {'session_id': 'firstchat'}}
model_with_memory.invoke([HumanMessage(content='hy i am pankaj kumawat')], config=config).content

"Hi Pankaj, yes, you are! It's good to hear from you again. How can I assist you today?"

In [44]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Pankaj, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--3a1f2f9a-81a2-4162-994b-76733d74c9f8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is **Pankaj Kumawat**. You told me that in your first message!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--88d5e684-c534-46e6-94ca-44360d79b119-0', usage_metadata={'input_tokens': 33, 'output_tokens': 19, 'total_tokens': 18

In [47]:
config={'configurable': {'session_id': 'fourth'}}

while True: 
    message=input('Write your query')
    if message=='end': 
        print('Have a great Day')
        break
    response=model_with_memory.invoke([HumanMessage(content=message)], config=config).content
    print(f'HumanMessage: {message}\nAIMessage: {response}')

HumanMessage: hy
AIMessage: Hi there! How can I help you today?
HumanMessage: my name is RAJA JI
AIMessage: Nice to meet you, Raja Ji! It's a pleasure to connect with you. How are you doing today?
HumanMessage: what is the output of 4 * 5
AIMessage: The output of 4 * 5 is **20**.
HumanMessage: what is my name
AIMessage: Your name is **RAJA JI**.
HumanMessage: what math operation i discussied
AIMessage: You discussed **multiplication**.
HumanMessage: okay add 10 in output
AIMessage: Okay, adding 10 to the previous output (which was 20):

20 + 10 = **30**
Have a great Day


In [48]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Pankaj, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--3a1f2f9a-81a2-4162-994b-76733d74c9f8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is **Pankaj Kumawat**. You told me that in your first message!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--88d5e684-c534-46e6-94ca-44360d79b119-0', usage_metadata={'input_tokens': 33, 'output_tokens': 19, 'total_tokens': 18

In [ ]:
store['fourth'].messages

[HumanMessage(content='hy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--045ce8ce-95d9-4583-a741-3e329bfb479e-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='my name is RAJA JI', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you, Raja Ji! It's a pleasure to connect with you. How are you doing today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7ab5d290-b895-46da-9837-eacd6f991c3f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 24, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='wha